# Intersection over Union
### Intersection / Union
* IoU > 0.5 "decent"
* IoU > 0.7 "pretty good"
* IoU > 0.9 "Almost perfect"
---

##### Box1 = [x1, y1, x2, y2]
##### Box2 = [x1, y1, x2, y2]

### Intersection (x1,y1,x2,y2)
* **Upper Left x1**: max(box1[0],box2[0])
* **Upper Left y1**: max(box1[1],box2[1])
* **Bottom Right x2**: min(box1[2],box2[2])
* **Bottom Right y2**: min(box1[3],box2[3])

* **Intersection Area**: abs(x1-x2) * abs(y1-y2)

### Union
* **Area**: Area(Box1) + Area(Box2) - Intersection

In [13]:
import numpy as np
import torch

In [1]:
def IoU(boxes_preds, boxes_labels, box_format="midpoint"):
    # boxes_preds - N x 4, where N is # boxes
    if box_format == 'midpoint':
        # [x,y,w,h]
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2 # N x 1
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 2:3] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 3:4] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 2:3] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 3:4] + boxes_labels[..., 3:4] / 2
        
    if(box_format == 'corners'):
        # [x1,y1,x2,y2]
        box1_x1 = boxes_preds[..., 0:1] # N x 1
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]
        
    
    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)
    
    
    # .clamp(0) is for the case when they DO NOT intersect. clamp(0) means set to 0 if it's less than 0
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))
    union = box1_area + box2_area - intersection
    
    print("intersection coordinates: [",x1.item(),y1.item(),x2.item(),y2.item(),"]")
    print("intersection:",intersection.item())
    print("union:",union.item())
    
    return (intersection / (union + 1e-6)).item()

In [26]:
IoU(torch.tensor([[1,0,3,2]]), torch.tensor([[2,1,5,4]]),'corners')

intersection coordinates: [ 2 1 3 2 ]
intersection: 1
union: 12


0.0833333283662796